In [57]:
!pip install shapely

In [58]:
#Libs

from kafka import KafkaProducer #Producer
import tweepy #get tweets
from json import dumps #read the tweet
from os import getenv #for stream
import statistics #count minutes
import random #generate random

from os import getenv
import tweepy
from kafka import KafkaProducer
from shapely.geometry import Polygon
import time

In [59]:
from shapely.geometry import Polygon

In [60]:
# My Tweeter Credentials

TWITTER_CONSUMER_API_KEY=''
TWITTER_CONSUMER_API_KEY_SECRET=''
TWITTER_ACCESS_TOKEN=''
TWITTER_ACCESS_TOKEN_SECRET=''

In [61]:
#TEST
##############
##############
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY, TWITTER_CONSUMER_API_KEY_SECRET) #initiale
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET) #credentials

api = tweepy.API(auth) #initiate API

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

Sinéad O’Connor calls out ‘evil’ Irish state and hospital after teenage son’s death https://t.co/CfvRfeOhm8
Ten of the best eye creams https://t.co/MKFgytGrbg
Canyon wall collapses onto tourist boats on lake in Brazil https://t.co/cAhPMUjlLC
From new year email catch-up to an oddly placed toilet. https://t.co/gzfMZF6AFQ
Apple’s Tim Cook received nearly $100 million in compensation last year, reaching his 10th anniversary in a year wh… https://t.co/abBcU54VDg
Urlauber-Familie: „Wir sind in Kitzbühel gefangen, können uns nicht freitesten!“
https://t.co/J0lTpOaHPQ https://t.co/27CyGUmWrj
Egyptian Palestinian activist Ramy Shaath has arrived in France after spending more than 2 years in detention in Eg… https://t.co/xfN0DhdrPs
Im vergangenen Jahr haben deutsche Firmen ein Drittel weniger #Plastikmüll exportiert. In #China und anderen asiati… https://t.co/9s6gWOLyke
Relative to its size, the snap of this shrimp packs a super-powerful punch. https://t.co/ckT0NNWBNa
Opinion by David Ignatius:

In [62]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name = "testy"

In [63]:
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY, TWITTER_CONSUMER_API_KEY_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [64]:
GEOBOX_City = []
GEOBOX_PARIS = [2.258707,48.816967,2.418009,48.902156]
GEOBOX_ROME = [12.234467,41.655642,12.85576,42.141028]
GEOBOX_Berlin = [13.095154,52.373752,13.737854,52.684539]
GEOBOX_Munich = [11.388505,48.045943,11.744187,48.220085]
GEOBOX_Vienna = [16.186311,48.118494,16.551925,48.328652]
GEOBOX_Zurich = [8.446361,47.332356,8.602573,47.422329]
GEOBOX_City = GEOBOX_PARIS + GEOBOX_ROME + GEOBOX_Berlin + GEOBOX_Munich + GEOBOX_Vienna + GEOBOX_Zurich

In [65]:
Dict_Cities = {"Paris": GEOBOX_PARIS, "Rome": GEOBOX_ROME, "Berlin": GEOBOX_Berlin, "Munich": GEOBOX_Munich, "Vienna": GEOBOX_Vienna, "Zurich": GEOBOX_Zurich}

In [66]:
Dict_Polygon = {}
for key, data in Dict_Cities.items():
    Dict_Polygon["%s" % key] = Polygon(
        [
            
                (data[0], data[1]),
                (data[0], data[3]),
                (data[2], data[1]),
                (data[2], data[3])
            
        ]
    )
    
print(Dict_Polygon)

{'Paris': <shapely.geometry.polygon.Polygon object at 0x7f4a26b63f70>, 'Rome': <shapely.geometry.polygon.Polygon object at 0x7f4a26b63fd0>, 'Berlin': <shapely.geometry.polygon.Polygon object at 0x7f4a26abdca0>, 'Munich': <shapely.geometry.polygon.Polygon object at 0x7f4a26abdb80>, 'Vienna': <shapely.geometry.polygon.Polygon object at 0x7f4a26abdbe0>, 'Zurich': <shapely.geometry.polygon.Polygon object at 0x7f4a26abdd60>}


In [67]:
class TwitterStreamer(tweepy.Stream):

        
    def on_status(self, status):
        
        valuepass = "{"+str(status.created_at)+", "+status.place.name+", "+status.text+"}"
        
        #Polygon aus den Daten erzeugen
        geo1 = status.place.bounding_box.coordinates[0][0][0]
        geo2 = status.place.bounding_box.coordinates[0][0][1]
        geo3 = status.place.bounding_box.coordinates[0][2][0]
        geo4 = status.place.bounding_box.coordinates[0][2][1]
        p1 = Polygon([(geo1, geo2), (geo1, geo4), (geo3, geo4), (geo3, geo2)])
        
        for key, value in Dict_Polygon.items():
            if (p1.intersects(value)):
                producer.send(topic_name, value = str.encode(key))
                
        print(valuepass)
             
        return True
    
    def on_error(self, status):
        print(status)

In [68]:
TwitterStreamer_Single_Tweets = TwitterStreamer(
    TWITTER_CONSUMER_API_KEY, TWITTER_CONSUMER_API_KEY_SECRET, TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

In [17]:
#Filter the Stream

TwitterStreamer_Single_Tweets.filter(locations=GEOBOX_City)

{2022-01-10 18:02:08+00:00, Berlin, @_xGoddess_Jenny https://t.co/iUSjgZ4yOZ}
{2022-01-10 18:02:10+00:00, Berlin, @aryscary Ook lekker, maar als brabander geef ik de voorkeur aan een worstenbroodje}
{2022-01-10 18:02:11+00:00, Roma, Alla fine avremo ragione NOI…tempo al tempo}
{2022-01-10 18:02:15+00:00, Berlin, Hat gerade ein Foto gepostet @ Kladow https://t.co/Hs6NPQHpzw}
{2022-01-10 18:02:15+00:00, Roma, ma anche a voi le FFP2 segano le orecchie?
avete dei rimedi?
RISP È IMPO!¡!¡!}
{2022-01-10 18:02:21+00:00, Aubervilliers, @ConsuladoFlaPB Navas escanteado!? Por esta razão que foi titular ontem contra Lyon no principal jogo do PSG no campeonato francês.}
{2022-01-10 18:02:30+00:00, Paris, @pinsardmathieu @lvariol @misterjbl @sandrousseau @RaphaelleRL @SachaHL1 @BakloutiFanny @annielahmer… https://t.co/qffTikZV6g}
{2022-01-10 18:02:35+00:00, Berlin, Bu beyefendiyi kulübümüze bulaştıranları da sevgiyle anıyorum.}
{2022-01-10 18:02:38+00:00, Roma, Sistemi di aereazione nelle scuole...f

KeyboardInterrupt: 